In [1]:
%matplotlib inline
import numpy as np
import preprocessing as prep
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.utils.model_zoo as model_zoo
from scipy.spatial.distance import cdist
from sklearn.metrics import roc_auc_score
from skimage.feature import corner_peaks,corner_harris,BRIEF,ORB
from skimage.color import rgb2gray
from skimage.filters import gaussian
from sklearn.utils import shuffle
import pdb

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [2]:
# Note: Overall  dataset consists of 400 images (100 per class among four classes). 
# We are using 320 images (80 per class) as Train set+oracle set and 80 images (20 per class) as Test set

# Control Parameters

Method_type = 3                                 # 0:Baseline accuracy computation (Entire dataset) 
                                                # 1: Random sampling  2:Max Entropy measure  3:Our method
Initial_dataset_size   = 104                      # Initial dataset size to sample using ORB descriptors
No_of_images_to_sample = 10                      # No of images to sample each AL iteration, common for all method types
No_of_images_entropy_method3 = 30               # No of images to sample initially from entropy measure (before using learnt feature vectors) for our method  
Distance_metric = 'euclidean'                   # Distance metric to compare model feature vectors
Create_initial_dataset = False                   # True:  Performs ORB descriptor technique (unless Load_descriptors = True) to create initial training and oracle sets from overall available dataset
                                                #        Use this to start the active learning process from stratch for a new method_type     
                                                # False: Loads most recent training and oracle sets stored in same directory
                                                #        and starts from the last Active learning iteration performed for a particular method_type  
Accuracy_array_name = 'ARSN_baseline/acc_MESS.npy'     # Numpy array storage name. This records the best test accuracy acheived during each AL iteration.
                                                # Change name for different method_type
AUC_array_name   = 'ARSN_baseline/AUC_MESS.npy'              # Stores the AUC score on test set for each AL iteration.Change name for different method_type
Load_descriptors = True                        # Loads precomputed descriptors for all dataset images.It is useful only when Create_initial_dataset==True  

# Network Training parameters
Epochs = 5                                # no of epochs to train model for each AL iteration
AL_iterations = 34                        # no of times to implement Active learning algorithm   
Batch_size = 32
train_from_stratch = True                 # Trains model from stratch for each AL iteration
learning_rate = 0.0001                    # Learning rate for deep CNN model
Img_resol     = 512                       # Input image resolution (Height == Width assumed) 
Availble_training_data_size = 444
Availble_test_data_size     = 85+85
No_classes = 2

path       = 'model/MESSIDOR_baseline.pt'


In [3]:
#Get images
orc_ARSN        = np.load('dataset/ARSN_oracle.npy', encoding='bytes')
orc_ARSN_label  = np.load('dataset/ARSN_oracle_label.npy', encoding='bytes')
test_ARSN       = np.load('dataset/ARSN_test.npy', encoding='bytes')
test_ARSN_label = np.load('dataset/ARSN_test_label.npy', encoding='bytes')

train_MESS       = np.load('dataset/MESSIDOR_train.npy', encoding='bytes')
train_MESS_label = np.load('dataset/MESSIDOR_train_label.npy', encoding='bytes')
test_MESS        = np.load('dataset/MESSIDOR_test.npy', encoding='bytes')
test_MESS_label  = np.load('dataset/MESSIDOR_test_label.npy', encoding='bytes')

#Preprocess images
orc_ARSN_prep    = np.transpose(orc_ARSN,  (0,3,1,2))
test_ARSN_prep   = np.transpose(test_ARSN, (0,3,1,2))
train_MESS_prep  = np.transpose(train_MESS,(0,3,1,2))
test_MESS_prep   = np.transpose(test_MESS, (0,3,1,2))
orc_ARSN_prep, test_ARSN_prep   = prep.clean(orc_ARSN_prep,test_ARSN_prep,Img_resol)
train_MESS_prep, test_MESS_prep = prep.clean(train_MESS_prep,test_MESS_prep,Img_resol)

#Weird
#orc_ARSN_prep, test_MESS_prep = prep.clean(orc_ARSN,test_MESS,Img_sol)


In [4]:
# Function definitions
def add_file_to_train(new_index):
    global X_train,Y_train,X_oracle,Y_oracle

    X_train_new = X_oracle[new_index]
    Y_train_new = Y_oracle[new_index]

    X_train = np.concatenate((X_train,X_train_new),axis=0)
    Y_train = np.concatenate((Y_train,Y_train_new),axis=0)

    X_oracle = np.delete(X_oracle,new_index,axis=0)
    Y_oracle = np.delete(Y_oracle,new_index,axis=0)

In [ ]:
def calculate_descriptors(X):
    descriptor_extractor = ORB(n_keypoints=500)
    Descriptors = []
    for i in range(len(X)):
        print ('Calculating ORB descriptor for image:{}'.format(i))
        Im  = np.asarray(X[i][:,:,:],dtype='float64')
        Max = np.amax(Im)
        Im = Im/Max
        Im = rgb2gray(Im)

        descriptor_extractor.detect_and_extract(Im)
        Temp = descriptor_extractor.descriptors
        Descriptors.append(np.asarray(np.round(np.average(Temp,axis=0)),dtype='int32'))

    Descriptors_matrix = np.zeros([len(X),256])
    for i in range(len(X)):
        Descriptors_matrix[i,:] = Descriptors[i] 

    return Descriptors_matrix


In [ ]:
def calculate_distance(X_oracle_descriptor,X_train_descriptor):
    Distances = np.zeros([len(X_oracle_descriptor),len(X_train_descriptor)])
    for i in range(len(X_oracle_descriptor)):
        Oracle = np.reshape(X_oracle_descriptor[i,:],(1,256))
        for j in range(len(X_train_descriptor)):
            Train = np.reshape(X_train_descriptor[j,:],(1,256))
            Distances[i,j] = cdist(Train,Oracle,'hamming')

    Distances = np.reshape(np.average(Distances,axis=1),(len(X_oracle_descriptor),))
    Sorted_distances = np.flip(np.sort(Distances,axis=0),axis=0)
    Sorted_indexes = np.flip(np.argsort(Distances,axis=0),axis=0)
    return Sorted_distances,Sorted_indexes

In [ ]:
def create_trainset():
    global Initial_dataset_size,X_oracle,X_oracle_descriptor,Y_oracle,Img_resol,No_classes
    Index = []
    Initial_sample = np.random.randint(0,len(X_oracle),1)
    Index.append(Initial_sample[0])

    X_train_descriptor =  np.reshape(X_oracle_descriptor[Initial_sample,:],(1,256))
    X_train = np.reshape(X_oracle[Initial_sample,:,:,:],(1,Img_resol,Img_resol,3))
    if No_classes==2:
        Y_train = np.reshape(Y_oracle[Initial_sample],(1,))
    else:
        Y_train = np.reshape(Y_oracle[Initial_sample,:],(1,No_classes))			

    X_oracle = np.delete(X_oracle,Initial_sample,axis=0)
    X_oracle_descriptor = np.delete(X_oracle_descriptor,Initial_sample,axis=0)
    Y_oracle = np.delete(Y_oracle,Initial_sample,axis=0)

    print ('Creating training dataset:')
    for i in range(Initial_dataset_size-1):
        Dist,Indexes = calculate_distance(X_oracle_descriptor, X_train_descriptor)

        Selected_Index = Indexes[0]
        Index.append(Selected_Index)

        Addition = np.reshape(X_oracle[Selected_Index,:,:,:],(1,Img_resol,Img_resol,3))
        X_train = np.concatenate((X_train,Addition),axis=0)

        Addition = np.reshape(X_oracle_descriptor[Selected_Index,:],(1,256))
        X_train_descriptor = np.concatenate((X_train_descriptor,Addition),axis=0)

        X_oracle = np.delete(X_oracle,Selected_Index,axis=0)
        X_oracle_descriptor = np.delete(X_oracle_descriptor,Selected_Index,axis=0)

        if No_classes==2:
            Addition = np.reshape(Y_oracle[Selected_Index],(1,))
        else:
            Addition = np.reshape(Y_oracle[Selected_Index],(1,No_classes))

        Y_train = np.concatenate((Y_train,Addition),axis=0)
        Y_oracle = np.delete(Y_oracle,Selected_Index,axis=0)

        X_train,Y_train = shuffle(X_train,Y_train,random_state=62)

    return X_train,Y_train,Index

In [ ]:
def compare_feature_vectors(oracle_feature_vectors,train_feature_vectors,Indices):
    global No_of_images_to_sample,Distance_metric
    indexes = []
    for i in range(No_of_images_to_sample):
        compared_distances = np.reshape(np.average(cdist(oracle_feature_vectors,train_feature_vectors,Distance_metric),axis=1),(len(oracle_feature_vectors),))  
        selected_distance_index = Indices[np.argmax(compared_distances)]

        Addition = np.reshape(oracle_feature_vectors[np.argmax(compared_distances),:],(1,oracle_feature_vectors.shape[1]))
        train_feature_vectors = np.concatenate((train_feature_vectors,Addition),axis=0)
        oracle_feature_vectors = np.delete(oracle_feature_vectors,np.argmax(compared_distances),axis=0)
        Indices = np.delete(Indices,np.argmax(compared_distances),axis=0)
        for j,r in enumerate(Indices):
            if r>selected_distance_index:
                    Indices[j] = r-1
        #print(selected_distance_index)
        indexes.append(selected_distance_index)
    add_file_to_train(indexes)

In [ ]:
def random_sample():
    global X_train,X_oracle,Y_train,Y_oracle,No_of_images_to_sample,Img_resol,No_classes

    Indice = np.arange(0,len(X_oracle),1)

    Positive_index = []
    for i in range(No_of_images_to_sample):
        I = np.random.randint(0,len(Indice),1)
        Positive_index.append(Indice[I])
        Indice = np.delete(Indice,I)

    if K.image_data_format() == 'channels_first':
        Addition = np.reshape(np.take(X_oracle,Positive_index,axis=0),(No_of_images_to_sample,3,Img_resol,Img_resol))
    else:
        Addition = np.reshape(np.take(X_oracle,Positive_index,axis=0),(No_of_images_to_sample,Img_resol,Img_resol,3)) 
    X_train = np.concatenate((X_train,Addition),axis=0)
    X_oracle = np.delete(X_oracle,Positive_index,axis=0)

    if No_classes==2:
        Addition = np.squeeze(np.take(Y_oracle,Positive_index,axis=0),axis=1)
    else:
        Addition = np.take(Y_oracle,Positive_index,axis=0)

    Y_train = np.concatenate((Y_train,Addition),axis=0)
    Y_oracle = np.delete(Y_oracle,Positive_index,axis=0)
   

In [ ]:
def compute_entropy(oracle_preds):
    global No_classes

    Entropy_measure = np.zeros([oracle_preds.shape[0],])
    if No_classes==2:
        for i,r in enumerate(oracle_preds):
            Entropy_measure[i] = -r*np.log2(r) + (1-r)*np.log2(1-r)
    else:
        for i in range(len(oracle_preds)):
            for j in oracle_preds[i,:]:
                if j!=0.0 and j!=1.0:
                    Entropy_measure[i] += -j*np.log2(j)

    return Entropy_measure

In [ ]:
class MedalTrainer:
    def __init__(self, model, index, batch_size, max_epochs=1, path='model.pt',
                 method=0,sample=0,sample_cross_entropy=0):
        global orc_ARSN_prep, orc_ARSN_label, test_ARSN_prep, test_ARSN_label, test_MESS_prep, test_MESS_label

        self.model      = model
        self.batch_size = batch_size
        self.train_ARSN      = orc_ARSN_prep[index]
        self.train_ARSN_label= orc_ARSN_label[index]
        orc_index = list(set(np.arange(orc_ARSN.shape[0]).tolist()) - set(index))
        self.orc_ARSN         = orc_ARSN_prep[orc_index]
        self.orc_ARSN_label   = orc_ARSN_label[orc_index]     
        self.test_ARSN        = test_ARSN_prep
        self.test_ARSN_label  = test_ARSN_label 
        self.test_MESS        = test_MESS_prep
        self.test_MESS_label  = test_MESS_label
        self.train_losses = []
        self.test_losses  = []
        self.test_acc     = []
        self.test_auc_score = []
        self.method = method
        self.sample = sample
        self.sample_cross_entropy = sample_cross_entropy
        self.epochs = 0
        self.prev_epoch = 0
        self.max_epochs = max_epochs
        self.path = path
        
        self.orc_ind_dict = {}
        for i in range(len(orc_index)):
            self.orc_ind_dict[str(i)] = orc_index[i]
        
        #model helper
        self.optimizer = torch.optim.Adam(model.parameters(),lr=0.0001, weight_decay=1e-6)
        self.criterion = nn.BCELoss()
        
    def train(self):
        self.model.train() # set to training mode
        epoch_loss = 0
        batch_num = 0
        index = np.random.choice(self.train_ARSN.shape[0], size=(self.train_ARSN.shape[0],), replace=False)
        for i in range(0, self.train_ARSN.shape[0]-self.batch_size, self.batch_size):
            inputs  = self.train_ARSN[index[i:i+self.batch_size]]
            targets = self.train_ARSN_label[index[i:i+self.batch_size]]
            loss = self.train_batch(inputs, targets)
            epoch_loss += loss
            batch_num += 1
            #if batch_num % 10 == 0:
                #print("At batch",batch_num)
                #print("Loss:",loss)
        epoch_loss = epoch_loss / batch_num
        self.epochs += 1
        if self.epochs == self.max_epochs:
            print('[TRAIN]  Epoch [%d/%d]   Loss: %.4f' % (self.epochs, self.max_epochs, epoch_loss))
        self.train_losses.append(epoch_loss)
        
        return epoch_loss

    def train_batch(self, inputs, targets):
        inputs  = torch.from_numpy(inputs).to(DEVICE).float()
        targets = torch.from_numpy(targets).to(DEVICE).float()
        outputs = torch.flatten(self.model(inputs))
        loss    = self.criterion(outputs,targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()
    
    def test(self):
        self.model.eval()
        test_loss = 0
        auc_score = 0
        test_acc  = 0
        batch_num = 0
        
        with torch.no_grad():
            for i in range(0, self.test_ARSN.shape[0], self.batch_size):
                inputs  = self.test_ARSN[i:i+self.batch_size]
                targets = self.test_ARSN_label[i:i+self.batch_size]
                loss, acc, r_sco = self.test_batch(inputs,targets)
                test_loss += loss
                test_acc  += acc
                auc_score += r_sco
                batch_num += 1
            test_loss /= batch_num
            auc_score  = 100* auc_score/batch_num
            test_acc   = 100* test_acc/(batch_num*self.batch_size)
            
            self.test_losses.append(test_loss)
            self.test_acc.append(test_acc)
            self.test_auc_score.append(auc_score)
            print('[Test]  Epoch [%d/%d]   Loss: %.4f Accuracy: %.4f AUC_Score: %.4f'
                      % (self.epochs, self.max_epochs, test_loss, test_acc, auc_score))
        return test_loss, test_acc, auc_score
            
    def test_batch(self, inputs, targets):
        inputs  = torch.from_numpy(inputs).to(DEVICE).float()
        targets = torch.from_numpy(targets).to(DEVICE).float()
        outputs = torch.flatten(self.model(inputs))
        loss    = self.criterion(outputs,targets) 
        predicted = torch.round(outputs.data)
        correct   = (predicted == targets).sum()
        auc_score = roc_auc_score(targets,outputs.data)
        
        return loss.item(), correct.item(), auc_score
    
    def fit(self):
        best_nll = 1e30
        best_acc = 0
        best_auc = 0
        for epoch in range(self.max_epochs):
            self.train()
            nll, acc, auc = self.test()
            if nll < best_nll:
                best_nll = nll
                best_acc = acc
                best_auc = auc
                self.save()
        return best_nll, best_acc, best_auc
    
    def get_new_index(self):
        with torch.no_grad():
            #Get Logits of oracle set
            inputs = torch.from_numpy(self.orc_ARSN[:len(self.orc_ARSN)//3]).to(DEVICE).float()
            orc_out_0 = self.model(inputs).cpu().numpy()
            inputs    = torch.from_numpy(self.orc_ARSN[len(self.orc_ARSN)//3:2*len(self.orc_ARSN)//3]).to(DEVICE).float()
            orc_out_1 = self.model(inputs).cpu().numpy()
            inputs    = torch.from_numpy(self.orc_ARSN[2*len(self.orc_ARSN)//3:]).to(DEVICE).float()
            orc_out_2 = self.model(inputs).cpu().numpy()
            orc_out = np.concatenate((orc_out_0,orc_out_1),0)
            orc_out = np.concatenate((orc_out,orc_out_2),0)
            
            #Get features of lowest entropy of oracle set
            entropy = compute_entropy(orc_out)
            low_entrop_ind = np.flip(np.argsort(entropy),axis=0)[:self.sample_cross_entropy]
            inputs      = torch.from_numpy(self.orc_ARSN[low_entrop_ind]).to(DEVICE).float()
            orc_feat      = self.model.forward(inputs,extract_feature=True).cpu().numpy()
            
            #Get features of train set
            inputs        = torch.from_numpy(self.train_ARSN[:len(self.train_ARSN)//2]).to(DEVICE).float() 
            train_feat_0  = self.model.forward(inputs,extract_feature=True).cpu().numpy()
            inputs        = torch.from_numpy(self.train_ARSN[len(self.train_ARSN)//2:]).to(DEVICE).float() 
            train_feat_1  = self.model.forward(inputs,extract_feature=True).cpu().numpy()
            train_feat    = np.concatenate((train_feat_0,train_feat_1), 0)
            
            compare_feature_vectors(orc_feat,train_feat,low_entrop_ind)
            
            new_ind = [self.orc_ind_dict[str(i)] for i in low_entrop_ind]
            #pdb.set_trace()
            
            return new_ind        
    
    def load(self,path='model/MESSIDOR_baseline.pt'):
        checkpoint = torch.load(path)
        self.prev_epoch = checkpoint['epoch']
        self.model.load_state_dict(checkpoint['state_dict'])
        self.model.cuda()
        print("Model Loaded!")
        
    def save(self,path='model/ARSN_active_learning_check.pt'):
        self.model.cpu()
        state = {'epoch': self.epochs+self.prev_epoch, 
                 'state_dict': self.model.state_dict()}
        torch.save(state, path)
        self.model.cuda()
        print("Model saved!")

In [ ]:
class Medal(nn.Module):
    def __init__(self, pretrain_inception, num_classes):
        super(Medal, self).__init__()
        self.inception = pretrain_inception
        self.linear_0        = nn.Linear(1000, 512)
        self.dropout         = nn.Dropout(0.5)
        self.linear_1        = nn.Linear(512, 128)
        self.linear_2        = nn.Linear(128, num_classes)
        self.relu            = nn.ReLU()
        self.sigmoid         = nn.Sigmoid()
        
        self.init_weight()
        
    def forward(self, images,extract_feature=False):
        if extract_feature:
            out = self.inception(images,extract_mixed_6=True)
            out = nn.AdaptiveAvgPool2d((1,1))(out)
            return torch.squeeze(out)
        
        out = self.inception(images)
        out = self.linear_0(out)
        out = self.dropout(out)
        out = self.relu(self.linear_1(out))
        out = self.linear_2(out)
        
        return self.sigmoid(out)
    
    def init_weight(self):
        self.linear_0.weight.data.uniform_(-0.1,0.1)
        self.linear_1.weight.data.uniform_(-0.1,0.1)
        self.linear_2.weight.data.uniform_(-0.1,0.1)
        
        self.linear_0.bias.data.fill_(0)
        self.linear_1.bias.data.fill_(0)
        self.linear_2.bias.data.fill_(0)
        

In [ ]:
# Load inception model
inception = models.inception_v3()

# Inception v3 pretrained weights
model_dir = '/afs/ece.cmu.edu/usr/sadom/Private/Active_Learning/pretrained_model'
model_urls = {'inception_v3_google': 
    'https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth',}
inception.load_state_dict(model_zoo.load_url(
    model_urls['inception_v3_google'],model_dir=model_dir))
inception.aux_logits = False

model   = Medal(inception,1)
model   = nn.DataParallel(model)
model   = model.to(DEVICE)

In [ ]:
X_oracle = orc_ARSN
Y_oracle = orc_ARSN_label
X_test   = test_MESS
Y_test   = test_MESS_label

if Method_type==0:
    print ('Training size: {}'.format(X_train.shape))
    print ('###########################################################')
    print ('Evaluating baseline performance...')
    Index = np.arange(len(X_oracle)).tolist()
    Epochs = 50
    trainer = MedalTrainer(model,Index,Batch_size,Epochs,path)
    loss, accuracy, auc_score = trainer.fit()
    print ('Baseline accuracy calculated:{}'.format(accuracy)) 
else:
    if Create_initial_dataset==True:
        if Load_descriptors==False: 
            X_oracle_descriptor = calculate_descriptors(X_oracle)
            np.save('dataset/ARSN_descriptor.npy',X_oracle_descriptor)
            print ('Descriptor array saved!!')
        else:
            X_oracle_descriptor = np.load('dataset/ARSN_descriptor.npy',encoding='bytes')

        X_train,Y_train,Index = create_trainset()

        np.save('ARSN_baseline/X_train.npy',X_train)
        np.save('ARSN_baseline/Y_train.npy',Y_train)
        np.save('ARSN_baseline/Index.npy', Index)
        np.save('ARSN_baseline/X_oracle.npy',X_oracle)
        np.save('ARSN_baseline/Y_oracle.npy',Y_oracle)
        
        Test_Accuracy = []
        AUC_score = []
        Class_count = []
        current_AL_iteration = 0

    else:
        X_train  = np.load('ARSN_baseline/X_train.npy', encoding='bytes')
        Y_train  = np.load('ARSN_baseline/Y_train.npy', encoding='bytes')
        X_oracle = np.load('ARSN_baseline/X_oracle.npy',encoding='bytes')
        Y_oracle = np.load('ARSN_baseline/Y_oracle.npy',encoding='bytes')
        Index    = np.load('ARSN_baseline/Index.npy',   encoding='bytes').tolist()

        Test_Accuracy = np.load(Accuracy_array_name,encoding='bytes').tolist()
        AUC_score = np.load(AUC_array_name,encoding='bytes').tolist()
        current_AL_iteration = np.load('ARSN_baseline/Current_AL_iteration.npy',encoding='bytes')[0]
        Class_count = list(np.load('ARSN_baseline/Class_count.npy',encoding='bytes'))

    best_auc = 86.46    
    print ('Train length:{}'.format(X_train.shape))
    for i in range(current_AL_iteration+1,AL_iterations+1):
        print ('############################################################')
        print ('Performing Active learning iteration {} for method {}'.format(i,Method_type))
        print ('Set sizes used for current AL iteration:')
        print ('Training set size:{}'.format(len(X_train)))
        print ('Oracle set size:{}'  .format(len(X_oracle)))

        trainer = MedalTrainer(model,Index,Batch_size,Epochs,path,Method_type,
                               No_of_images_to_sample,No_of_images_entropy_method3)
        trainer.load()
        loss, accuracy, auc_score = trainer.fit()
        if auc_score > best_auc:
            best_auc = auc_score
            trainer.save(path='model/ARSN_active_learning.pt')
        
        AUC_score.append(auc_score)        
        Test_Accuracy.append(accuracy)
        np.save(Accuracy_array_name,np.array(Test_Accuracy))        
        np.save(AUC_array_name,np.array(AUC_score))
        np.save('ARSN_baseline/Current_AL_iteration.npy',np.array([i]))
        np.save('ARSN_baseline/Class_count.npy',np.asarray(Class_count))
        
        Positive_count = list(Y_train).count(1)
        Negative_count = list(Y_train).count(0)     
        Class_count.append([Positive_count,Negative_count])
        print ('Positive Class count:{}'.format(Positive_count))
        print ('Negative Class count:{}'.format(Negative_count))
        print ('Best Test accuracy achieved for this AL iteration:{0} AUC score:{1}'.format(accuracy,auc_score))

        new_ind = trainer.get_new_index()
        for ind in new_ind:
            Index.append(ind)
        
        np.save('ARSN_baseline/Index.npy', Index)
        np.save('ARSN_baseline/X_train.npy',X_train)
        np.save('ARSN_baseline/Y_train.npy',Y_train)
        np.save('ARSN_baseline/X_oracle.npy',X_oracle)
        np.save('ARSN_baseline/Y_oracle.npy',Y_oracle)
        

Train length:(264, 512, 512, 3)
############################################################
Performing Active learning iteration 19 for method 3
Set sizes used for current AL iteration:
Training set size:264
Oracle set size:187
Model Loaded!
[Test]  Epoch [1/5]   Loss: 0.7895 Accuracy: 75.0000 AUC_Score: 77.1219
Model saved!
[Test]  Epoch [2/5]   Loss: 0.6443 Accuracy: 76.5625 AUC_Score: 71.2539
Model saved!
[Test]  Epoch [3/5]   Loss: 0.8741 Accuracy: 76.5625 AUC_Score: 70.6462
[Test]  Epoch [4/5]   Loss: 1.0224 Accuracy: 75.0000 AUC_Score: 71.3735
[TRAIN]  Epoch [5/5]   Loss: 0.0298
[Test]  Epoch [5/5]   Loss: 1.3626 Accuracy: 75.5208 AUC_Score: 69.2612
Positive Class count:236
Negative Class count:28
Best Test accuracy achieved for this AL iteration:76.5625 AUC score:71.25385802469137
############################################################
Performing Active learning iteration 20 for method 3
Set sizes used for current AL iteration:
Training set size:274
Oracle set size:177
Mod